# Data Wrangling & Standardization

In [41]:
#load in necessary packages
import pandas as pd
import numpy as np
pd.options.display.max_columns = 100
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import seaborn as sns
import datetime, tqdm

import math

from scipy.spatial import Voronoi, voronoi_plot_2d

In [53]:
games = pd.read_csv('Data/games.csv')
pff = pd.read_csv('Data/pffScoutingData.csv')
players = pd.read_csv('Data/players.csv')
plays = pd.read_csv('Data/plays.csv')
week1 = pd.read_csv('Data/week1.csv')

In [54]:
games = games.drop(['gameTimeEastern', 'gameDate', 'week', 'season'], axis = 1)
players = players[['nflId', 'displayName']]
train = week1.merge(games, how = 'left', on = 'gameId')
train = train.merge(players, how = 'left', on = 'nflId')
train = train.merge(plays, how = 'left', on = ['gameId', 'playId'])

In [55]:
train.head()

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event,homeTeamAbbr,visitorTeamAbbr,displayName,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,gameClock,preSnapHomeScore,preSnapVisitorScore,passResult,penaltyYards,prePenaltyPlayResult,playResult,foulName1,foulNFLId1,foulName2,foulNFLId2,foulName3,foulNFLId3,absoluteYardlineNumber,offenseFormation,personnelO,defendersInBox,personnelD,dropBackType,pff_playAction,pff_passCoverage,pff_passCoverageType
0,2021090900,97,25511.0,1,2021-09-10T00:26:31.100,12.0,TB,right,37.77,24.22,0.29,0.30,0.03,165.16,84.99,None,TB,DAL,Tom Brady,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,13:33,0,0,I,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man
1,2021090900,97,25511.0,2,2021-09-10T00:26:31.200,12.0,TB,right,37.78,24.22,0.23,0.11,0.02,164.33,92.87,None,TB,DAL,Tom Brady,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,13:33,0,0,I,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man
2,2021090900,97,25511.0,3,2021-09-10T00:26:31.300,12.0,TB,right,37.78,24.24,0.16,0.10,0.01,160.24,68.55,None,TB,DAL,Tom Brady,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,13:33,0,0,I,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man
3,2021090900,97,25511.0,4,2021-09-10T00:26:31.400,12.0,TB,right,37.73,24.25,0.15,0.24,0.06,152.13,296.85,None,TB,DAL,Tom Brady,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,13:33,0,0,I,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man
4,2021090900,97,25511.0,5,2021-09-10T00:26:31.500,12.0,TB,right,37.69,24.26,0.25,0.18,0.04,148.33,287.55,None,TB,DAL,Tom Brady,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,13:33,0,0,I,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man


In [56]:
train['toLeft'] = train.playDirection == 'left'

In [57]:
train['dir_rad'] = np.mod(90 - train.dir, 360) * math.pi/180

In [58]:
#orient the offense to always go left
train['teamOnOffense'] = "home"
train.loc[train.possessionTeam != train.homeTeamAbbr, 'teamOnOffense'] = "away"
train['isOnOffense'] = train.team == train.teamOnOffense # Is player on offense?
train['yardLine_std'] = 100 - train.yardlineNumber
train.loc[train.yardlineSide.fillna('') == train.possessionTeam,  
          'yardLine_std'
         ] = train.loc[train.yardlineSide.fillna('') == train.possessionTeam,  
          'yardlineNumber']
train['x_std'] = train.x
train.loc[train.toLeft, 'x_std'] = 120 - train.loc[train.toLeft, 'x'] 
train['y_std'] = train.y
train.loc[train.toLeft, 'y_std'] = 160/3 - train.loc[train.toLeft, 'y'] 
#train['Orientation_std'] = -90 + train.Orientation
#train.loc[train.ToLeft, 'Orientation_std'] = np.mod(180 + train.loc[train.ToLeft, 'Orientation_std'], 360)
train['dir_std'] = train.dir_rad
train.loc[train.toLeft, 'dir_std'] = np.mod(np.pi + train.loc[train.toLeft, 'dir_rad'], 2*np.pi)

## Define Function to Prep Data

In [1]:
def data_prep():

SyntaxError: unexpected EOF while parsing (<ipython-input-1-89dcce483681>, line 1)